In [2]:
# default_exp database_proxy

In [1]:
import pandas as pd
import sqlite3
import time

# Database Proxy

Handles Database connection and queries

Tables organization: 

Entities_[letter or not_alpha]
entity_id: Primary key, Unique 
label: name of the entity
description: Short description 

Properties_relations Table
entity_id: relationship of  
property_id: Key for properties table
related_entity_id

Aliases Table
entity_id 
alias: Other labels. 

Properties Table
property_id Unique primary key  
label:Description of the Property 

In [ ]:
class WikiDatabase:
    conn = None

    def __init__(self):
        try:
            self.conn = sqlite3.connect('wiki_database2.db')
        except Exception as e:
            print(e)
            exit(-1)

    def exit_procedure(self):
        self.conn.close()
        exit(-1)

    @staticmethod
    def get_table_name(entity_label):
        """
        @:param self:
        @:param label of the entity for which you need to
        :rtype: string
        @:return name of the table where entity is stored
        """
        if entity_label[0].isalpha():
            return entity_label[[0]].lower()
        else:
            return 'not_alpha'

    # TODO: Test the return value
    # TODO: change to check in the entities table
    def get_similar_entities_id(self, label):
        """
        Returns a list of entities id where the label matches the entity_label name
        :param self:
        :type self:
        :param label:
        :type label:
        :return: list of all the entities id and label where the label matched
        :rtype: Array containing all similar entities
        """
        entities_id = None
        try:
            entities_id = pd.read_sql_query(
                "SELECT * FROM Entities WHERE label  LIKE \"%{}%\";".format(label),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        return entities_id.tolist()

    def get_specific_entity(self, label):
        """
        Search for the id, label and description of an specific identity
        :param label: string of the label to look for
        :type label: string
        :return: id, label and description of the label
        :rtype: pandas.core.series.Series
        """
        entity_id = None
        table_name = self.get_table_name(label)
        try:
            entity_id = pd.read_sql_query(
                "SELECT * FROM Entities_{} WHERE label = \"{}\";".format(table_name, label),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        return entity_id.iloc[0]

    def get_entity_properties(self, entity_id):
        """
        Search for all properties of the entity
        Return all the
        :param entity_id: id of a entity
        :type entity_id: str
        :return: id's of the properties and related entities
        :rtype:
        """
        entity_properties = None
        try:
            entity_properties = pd.read_sql_query(
                "SELECT property_id, related_entity_id FROM Properties_relations WHERE entity_id = \"{}\";".format(entity_id),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        return entity_properties

    def get_entity_info(self, entity_id):

        relation_label = None
        try:
            relation_label = pd.read_sql_query(
                "SELECT label, description FROM Entities WHERE entity_id = \"{}\"".format(entity_id),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        if relation_label.empty:
            return None
        return relation_label.iloc[0]["label"]

    def get_property_label(self, property_id):
        """
        :param property_id: id of the property to look for
        :type property_id:
        :return: label of the given id, None if the id does not exist
        :rtype:
        """
        entity_label = None
        try:
            entity_label = pd.read_sql_query(
                "SELECT label FROM Properties WHERE property_id = \"{}\"".format(property_id),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        if entity_label.empty:
            return None
        return entity_label.iloc[0]["label"]

    def get_relation_string(self, entity_label, property_id, related_entity_id):
        """
        :param entity_label: if
        :type entity_label: string
        :param property_id:
        :type property_id: string
        :param related_entity_id:
        :type related_entity_id: string
        :return:
        :rtype:
        """
        entity_label = None
        related_entity_label = None
        property_name = None
        try:
            # entity_label = get_entity_label(entity_id)
            related_entity_label = self.get_entity_label(related_entity_id)
            property_name = self.get_property_label(property_id)
        except Exception as e:
            print(e)
            self.exit_precedure()
        if entity_label is None or  related_entity_label is None or property_name is None:
            return None
        return "{} {} {}".format(entity_label, property_name, related_entity_label)

    # def __del__(self):
    #     self.conn.close()

# Testing

In [ ]:
# wiki = WikiDatabase()
# start_time = time.time()
# result = wiki.get_specific_entity('Cristiano Ronaldo')
# print('Query tool : {}'.format(time.time()- start_time))
# print(type(result))
# print(result)
# wiki.exit_procedure()
